In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
import logging
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
import matplotlib.pyplot as plt

# Create a logger
logging.basicConfig(format="%(asctime)s %(levelname)s: %(message)s")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### Constants

In [2]:
# Directories
processed_dir = "data"

sample_sheets_dir = "sample_sheets"

cnv_data_dir = "/mnt/d/Documents/Data/TCGA/CNV"
dna_methylation_data_dir = "/mnt/d/Documents/Data/TCGA/DNA_Methylation"
gene_expression_data_dir = "/mnt/d/Documents/Data/TCGA/gene_expression"
mirna_data_dir = "/mnt/d/Documents/Data/TCGA/miRNA"

# Statuses
cnv_process_status = "processing"
dna_methylation_process_status = "processing"
gene_expression_process_status = "processing"
miRNA_process_status = "processing"

### Preprocess clinical data

In [3]:
df_clinical = pd.read_csv(f"{sample_sheets_dir}/gdc_clinical.tsv", sep="\t")
# df.columns.to_list()

In [4]:
# Select only some fields
selected_cols = [
    "case_submitter_id",
    "age_at_index",
    "days_to_death",
    "days_to_last_follow_up",
    "morphology",
    "ethnicity",
    "gender",
    "race",
    "vital_status",
    # "year_of_birth",
    # "year_of_death",
    "ajcc_pathologic_m",
    "ajcc_pathologic_n",
    "ajcc_pathologic_stage",
    "primary_diagnosis",
    "treatment_or_therapy",
    "treatment_type",
]

df_clinical = df_clinical[selected_cols]

# Replace '--' with NaN
df_clinical = df_clinical.replace("'--", np.nan)

logger.info(df_clinical.shape)
df_clinical

2025-01-14 13:53:42,929 INFO: (1107, 15)


,case_submitter_id,age_at_index,days_to_death,days_to_last_follow_up,morphology,ethnicity,gender,race,vital_status,ajcc_pathologic_m,ajcc_pathologic_n,ajcc_pathologic_stage,primary_diagnosis,treatment_or_therapy,treatment_type
0,TCGA-62-A471,64,NaN,1246.0,8140/3,not hispanic or latino,male,white,Alive,M0,N1,Stage IIB,"Adenocarcinoma, NOS",yes,"Pharmaceutical Therapy, NOS"
1,TCGA-62-A471,64,NaN,1246.0,8140/3,not hispanic or latino,male,white,Alive,M0,N1,Stage IIB,"Adenocarcinoma, NOS",no,"Radiation Therapy, NOS"
2,TCGA-67-3773,84,NaN,427.0,8140/3,not hispanic or latino,female,white,Alive,M0,N0,Stage IB,"Adenocarcinoma, NOS",not reported,"Radiation Therapy, NOS"
3,TCGA-67-3773,84,NaN,427.0,8140/3,not hispanic or latino,female,white,Alive,M0,N0,Stage IB,"Adenocarcinoma, NOS",not reported,"Pharmaceutical Therapy, NOS"
4,TCGA-17-Z038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,TCGA-55-7570,60,NaN,824.0,8140/3,not hispanic or latino,male,black or african american,Alive,MX,N0,Stage IA,"Adenocarcinoma, NOS",no,"Pharmaceutical Therapy, NOS"
1103,TCGA-78-7146,71,173,NaN,8255/3,not reported,female,white,Dead,M0,N2,Stage IIIA,Adenocarcinoma with mixed subtypes,no,"Radiation Therapy, NOS"
1104,TCGA-78-7146,71,173,NaN,8255/3,not reported,female,white,Dead,M0,N2,Stage IIIA,Adenocarcinoma with mixed subtypes,no,"Pharmaceutical Therapy, NOS"
1105,TCGA-44-3398,77,NaN,1163.0,8140/3,not hispanic or latino,female,white,Alive,M0,N0,Stage IA,"Adenocarcinoma, NOS",no,"Pharmaceutical Therapy, NOS"


In [5]:
# Add new column for Pharmaceutical Therapy, NOS
# Add new column for Radiation Therapy, NOS
df_clinical['pharmaceutical_treatment'] = ((df_clinical['treatment_type'] == 'Pharmaceutical Therapy, NOS') & (df_clinical["treatment_or_therapy"] == "yes")).astype(int)
df_clinical['radiation_treatment'] = ((df_clinical['treatment_type'] == 'Radiation Therapy, NOS') & (df_clinical["treatment_or_therapy"] == "yes")).astype(int)

In [6]:
# Group by 'case_submitter_id' and aggregate
df_clinical = df_clinical.groupby('case_submitter_id').agg({
    "age_at_index": "first",
    "days_to_death": "first",
    "days_to_last_follow_up": "first",
    "ethnicity": "first",
    "gender": "first",
    "race": "first",
    "age_at_index": "first",
    "morphology": "first",
    "vital_status": "first",
    "ajcc_pathologic_m": "first",
    "ajcc_pathologic_n": "first",
    "ajcc_pathologic_stage": "first",
    "primary_diagnosis": "first",
    "pharmaceutical_treatment": "max",             # Update to 1 if any row has 1
    "radiation_treatment": "max"                   # Update to 1 if any row has 1
}).reset_index()

# Update 'treatment_type' based on 'treatment_or_therapy'
df_clinical['treatment_type'] = np.where(df_clinical['pharmaceutical_treatment'] == 1, 'Pharmaceutical Therapy, NOS', 'None')
df_clinical['treatment_type'] = np.where(df_clinical['radiation_treatment'] == 1, 'Radiation Therapy, NOS', df_clinical['treatment_type'])

# Remove rows where 'vital_status' is NaN
df_clinical = df_clinical.dropna(subset=['vital_status'])

# Remove rows where 'days_to_death' or 'days_to_last_follow_up' is NaN
df_clinical = df_clinical[(df_clinical["days_to_death"].notna()) | (df_clinical["days_to_last_follow_up"].notna())]

# Convert vital_status to a binary event indicator (1 = Dead, 0 = Alive)
df_clinical['event'] = (df_clinical['vital_status'] == 'Dead').astype(int)

# Get 'days_to_event'
df_clinical['days_to_event'] = np.where((df_clinical['days_to_death'].notna()) & (df_clinical['event'] == 1), df_clinical['days_to_death'], df_clinical['days_to_last_follow_up']).astype(float)

df_clinical = df_clinical[df_clinical['days_to_event'] > 0]

# Rename some columns
df_clinical = df_clinical.rename(columns={'case_submitter_id': 'case_id'})

# Drop some columns
df_clinical = df_clinical.drop(columns=['pharmaceutical_treatment', 'radiation_treatment'])

logger.info(f"Total samples: {df_clinical.shape[0]}")
logger.info(f"Total clincal features: { df_clinical.shape[1]}")

df_clinical

2025-01-14 13:53:42,964 INFO: Total samples: 509
2025-01-14 13:53:42,964 INFO: Total clincal features: 16


,case_id,age_at_index,days_to_death,days_to_last_follow_up,ethnicity,gender,race,morphology,vital_status,ajcc_pathologic_m,ajcc_pathologic_n,ajcc_pathologic_stage,primary_diagnosis,treatment_type,event,days_to_event
1,TCGA-05-4245,81,None,730.0,not reported,male,not reported,8140/3,Alive,M0,N2,Stage IIIA,"Adenocarcinoma, NOS",None,0,730.0
2,TCGA-05-4249,67,None,1523.0,not reported,male,not reported,8140/3,Alive,M0,N0,Stage IB,"Adenocarcinoma, NOS",None,0,1523.0
3,TCGA-05-4250,79,121,None,not reported,female,not reported,8140/3,Dead,M0,N1,Stage IIIA,"Adenocarcinoma, NOS",None,1,121.0
4,TCGA-05-4382,68,None,607.0,not reported,male,not reported,8255/3,Alive,M0,N0,Stage IB,Adenocarcinoma with mixed subtypes,"Radiation Therapy, NOS",0,607.0
5,TCGA-05-4384,66,None,426.0,not reported,male,not reported,8255/3,Alive,M0,N2,Stage IIIA,Adenocarcinoma with mixed subtypes,"Radiation Therapy, NOS",0,426.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,TCGA-NJ-A55O,56,None,13.0,not hispanic or latino,female,white,8480/3,Alive,M0,N1,Stage IIA,Mucinous adenocarcinoma,None,0,13.0
581,TCGA-NJ-A55R,67,None,603.0,not hispanic or latino,male,white,8230/3,Alive,MX,N0,Stage IA,"Solid carcinoma, NOS",None,0,603.0
582,TCGA-NJ-A7XG,49,None,617.0,not hispanic or latino,male,black or african american,8140/3,Alive,M0,N1,Stage IIIA,"Adenocarcinoma, NOS","Pharmaceutical Therapy, NOS",0,617.0
583,TCGA-O1-A52J,74,1798,None,not hispanic or latino,female,white,8140/3,Dead,MX,N0,Stage IA,"Adenocarcinoma, NOS",None,1,1798.0


In [7]:
# Export data
df_clinical.to_csv(f"{processed_dir}/clincal.tsv", sep="\t")

# Get list of cases
list_cases = df_clinical["case_id"].to_list()

### Preprocess CNV data

In [8]:
df_cnv_sample_sheet = pd.read_csv(f"{sample_sheets_dir}/gdc_cnv_sample_sheet.tsv", sep="\t")

In [9]:
# Remove duplication cases
df_cnv_sample_sheet = df_cnv_sample_sheet.drop_duplicates(subset="Case ID", keep='first').reset_index(drop=True)
df_cnv_sample_sheet.shape
df_cnv_sample_sheet.head(5)

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,95e5fe95-789d-4223-b9cc-a16c7fd683c1,TCGA-LUAD.6ab7dce7-f54d-4f9e-b232-7f094e8bf05d...,Copy Number Variation,Gene Level Copy Number,TCGA-LUAD,TCGA-38-4630,TCGA-38-4630-01A,Primary Tumor
1,8c9ac2ea-6123-445c-8bd4-b0ea7c681edf,TCGA-LUAD.6e3bfcae-bea1-46b3-afee-df2e3a344cd1...,Copy Number Variation,Gene Level Copy Number,TCGA-LUAD,TCGA-50-5932,TCGA-50-5932-01A,Primary Tumor
2,b09d319f-7fba-4677-9aa2-fb2666643884,TCGA-LUAD.1afb48ce-59a8-4130-8ff6-45d29405087e...,Copy Number Variation,Gene Level Copy Number,TCGA-LUAD,TCGA-J2-A4AD,TCGA-J2-A4AD-01A,Primary Tumor
3,74cb107f-adc9-4aee-8381-c95831dbd4ea,TCGA-LUAD.d0597d14-df0c-413b-888f-53597d1fb61a...,Copy Number Variation,Gene Level Copy Number,TCGA-LUAD,TCGA-38-4625,TCGA-38-4625-01A,Primary Tumor
4,b6b74adb-31e0-4ecf-9a65-aa8a1babc46b,TCGA-LUAD.780c0b4e-e589-4cc3-b0da-ea99e908afdd...,Copy Number Variation,Gene Level Copy Number,TCGA-LUAD,TCGA-55-7284,TCGA-55-7284-01B,Primary Tumor


In [10]:
# Check for duplication cases
value_counts = df_cnv_sample_sheet['Case ID'].value_counts()
df_cnv_sample_sheet[df_cnv_sample_sheet['Case ID'].isin(value_counts[value_counts > 1].index)][["File ID", "File Name", "Case ID"]]

,File ID,File Name,Case ID


In [11]:
# Get cnv data of case exist in clinical data
df_cnv_sample_sheet = df_cnv_sample_sheet[df_cnv_sample_sheet["Case ID"].isin(list_cases)]
df_cnv_sample_sheet.shape

(493, 8)

In [ ]:
# Loop through all rows
case_data = []
if cnv_process_status != "processed":
    for index, row in df_cnv_sample_sheet.iterrows():
        cnv_data_file_path = f"{cnv_data_dir}/{row['File ID']}/{row['File Name']}"
        case_id = row["Case ID"]
        # Check if file exist
        if os.path.exists(cnv_data_file_path):
            # logger.info(f"Processing case: {row['Case ID']} at {cnv_data_file_path}")
            df_cnv_data = pd.read_csv(cnv_data_file_path, sep="\t")
            # print(df_cnv_data)
            df_cnv_data = df_cnv_data[df_cnv_data['copy_number'].notna()][['chromosome','gene_name', 'copy_number']]
            df_cnv_data = df_cnv_data.groupby('gene_name', as_index=False)['copy_number'].mean()
            
            # value_counts = df_cnv_data['gene_name'].value_counts()
            # print(df_cnv_data[df_cnv_data['gene_name'].isin(value_counts[value_counts > 1].index)][["gene_name", "copy_number"]])
            df_cnv_case_data = df_cnv_data.set_index('gene_name').T
            df_cnv_case_data['case_id'] = case_id
            df_cnv_case_data = df_cnv_case_data.set_index('case_id')
       
            case_data.append(df_cnv_case_data)
        else:
            logger.error(f"File {cnv_data_file_path} not exists!")
    

In [ ]:
# Merge all cases
if cnv_process_status != "processed":
    df_cnv_data = pd.concat(case_data, axis=0)
    df_cnv_data.columns.name = None
    df_cnv_data.to_csv(f"{processed_dir}/cnv_raw.tsv", sep="\t")
else:
    df_cnv_data = pd.read_csv(f"{processed_dir}/cnv_raw.tsv", sep="\t")
    
logger.info(df_cnv_data.shape)

In [ ]:
# Calculate the percentage of missing values per column
missing_percentage = df_cnv_data.isnull().mean() * 100

# Summarize the number of columns in different ranges of missing values
summary = {
    '0-10%': (missing_percentage <= 10).sum(),
    '10-30%': ((missing_percentage > 10) & (missing_percentage <= 30)).sum(),
    '30-50%': ((missing_percentage > 30) & (missing_percentage <= 50)).sum(),
    '50-100%': (missing_percentage > 50).sum()
}
logger.info("Summary of missing value percentages:")
logger.info(summary)

# Define a threshold for missing values
threshold = 0.5  # 50% threshold

# Drop columns with more than 50% missing values
df_cnv_data_processed = df_cnv_data.loc[:, missing_percentage <= (threshold * 100)]
logger.info(df_cnv_data_processed.shape)

# Fill NA with CN = 2 which is normal for human
df_cnv_data_processed = df_cnv_data_processed.fillna(2)

# Add prefix for every field
df_cnv_data_processed = df_cnv_data_processed.add_prefix("CNV_gene_")

logger.info(df_cnv_data_processed.head())

In [ ]:
# Export data
if cnv_process_status != "processed":
    df_cnv_data_processed.to_csv(f"{processed_dir}/cnv.tsv", sep="\t")
else:
    df_cnv_data_processed = pd.read_csv(f"{processed_dir}/cnv.tsv", sep="\t")

### Preprocess DNA Methylation data

In [ ]:
# Load DNA methylation sample sheet
df_dna_methylation_sample_sheet = pd.read_csv(f"{sample_sheets_dir}/gdc_dna_methylation_sample_sheet.tsv", sep="\t")

In [ ]:
# Check for duplication cases
value_counts = df_dna_methylation_sample_sheet['Case ID'].value_counts()
df_dna_methylation_sample_sheet[df_dna_methylation_sample_sheet['Case ID'].isin(value_counts[value_counts > 1].index)]

In [ ]:
# Remove duplication cases
logger.info(df_dna_methylation_sample_sheet.shape)

# Keeping only Primary Tumor
df_dna_methylation_sample_sheet = df_dna_methylation_sample_sheet[df_dna_methylation_sample_sheet['Sample Type'] == 'Primary Tumor']

# Keep the first Case ID only
df_dna_methylation_sample_sheet = df_dna_methylation_sample_sheet.drop_duplicates(subset="Case ID", keep='first').reset_index(drop=True)

logger.info(df_dna_methylation_sample_sheet.shape)
df_dna_methylation_sample_sheet.head(5)

In [ ]:
# Loop through all rows
dna_methylation_case_data = []
dna_methylation_column_names = ["id", "value"]

logger.info(df_dna_methylation_sample_sheet.shape)

for index, row in df_dna_methylation_sample_sheet.iterrows():
    dna_methylation_data_file_path = f"{dna_methylation_data_dir}/{row['File ID']}/{row['File Name']}"
    case_id = row["Case ID"]
    logger.info(f"At: {index}.")
    # Check if file exist
    if os.path.exists(dna_methylation_data_file_path):
        # logger.info(f"Processing case: {row['Case ID']} at {dna_methylation_data_file_path}")
        df_dna_methylation_data = pd.read_csv(dna_methylation_data_file_path, header=None, names=dna_methylation_column_names, sep="\t")
        
        # Include only cg probes
        df_dna_methylation_data = df_dna_methylation_data[df_dna_methylation_data['id'].astype(str).str.startswith('cg')]

        # Exclude NaN value
        df_dna_methylation_data = df_dna_methylation_data[df_dna_methylation_data['value'].notna()]
    
        df_dna_methylation_case_data = df_dna_methylation_data.set_index('id').T
        df_dna_methylation_case_data['case_id'] = case_id
        df_dna_methylation_case_data = df_dna_methylation_case_data.set_index('case_id')
   
        dna_methylation_case_data.append(df_dna_methylation_case_data)
        
        logger.info(f"At: {index}. Shape: {df_dna_methylation_data.shape}")
    else:
        logger.error(f"File {cnv_data_file_path} not exists!")
logger.info("Done")

In [ ]:
# Merge all cases
if dna_methylation_process_status != "processed":
    df_dna_methylation_data = pd.concat(dna_methylation_case_data, axis=0)
    df_dna_methylation_data.columns.name = None
    df_dna_methylation_data.to_csv(f"{processed_dir}/dna_methylation_raw.tsv", sep="\t")
else:
    df_dna_methylation_data = pd.read_csv(f"{processed_dir}/dna_methylation_raw.tsv", sep="\t")
    
logger.info(df_dna_methylation_data.shape)

In [ ]:
# Calculate the percentage of missing values per column
missing_percentage = df_dna_methylation_data.isnull().mean() * 100

# Summarize the number of columns in different ranges of missing values
summary = {
    '0-10%': (missing_percentage <= 10).sum(),
    '10-30%': ((missing_percentage > 10) & (missing_percentage <= 30)).sum(),
    '30-50%': ((missing_percentage > 30) & (missing_percentage <= 50)).sum(),
    '50-100%': (missing_percentage > 50).sum()
}
logger.info("Summary of missing value percentages:")
logger.info(summary)

# Define a threshold for missing values
threshold = 0.2  # 20% threshold

# Drop columns with more than 50% missing values
df_dna_methylation_data_processed = df_dna_methylation_data.loc[:, missing_percentage <= (threshold * 100)]
logger.info(df_dna_methylation_data_processed.shape)

# Impute missing values with the mean of each probe
df_dna_methylation_data_processed = df_dna_methylation_data_processed.fillna(df_dna_methylation_data_processed.mean())

# Add prefix for every field
df_dna_methylation_data_processed = df_dna_methylation_data_processed.add_prefix("DNA_meth_")

logger.info(df_dna_methylation_data_processed.head())